In [7]:
#KIA_FAQ
# pip install requests beautifulsoup4
import requests
from bs4 import BeautifulSoup

url = "https://www.kia.com/content/cq:graphql/global/endpoint.json"

# questions
a_selector = ".cmp-accordion__title"
# user-agent
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'

def get_question_list():
    """
    FAQ questions 크롤링.

    aguments
    return
        DataFrame: 조회결과들을 담은 DataFrame(표)
    raise
        Exception: 처리 실패시 발생
    """
    # 1. 요청
    res = requests.post(url, headers={"user-agent":user_agent})
    params = {
        
        'query' : '''query($tag:String) {
            faqList (
                filter: {
                  tags: {
                    _expressions:  {
                      	value:$tag
                      	_operator:EQUALS
                    }
                  }
                  _path: {
                    _expressions: [
                        {
                            value: "/content/dam/kwp/kr/ko"
                           _operator: STARTS_WITH
                        }
                    ]
                    }
                }
        
            ) {
                items {
                    _path
                    question
                    answer {
                        html
                        plaintext
                    }
                    important
                    sortorder
                    created
                    assets {
                        ... on AssetModel {
                          title
                          asset {
                              __typename
                              ... on ImageRef {
                                _path
                              }
                          }
                          mobileAsset{
                              __typename
                              ... on ImageRef {
                                _path
                              }
                          }
                          imageBorder
                        }
                    }
                    tags
                }
            }
        }

    ''',
        'variables' : '{"tag": "kwp:kr/faq/top10"}'
    }
    res = requests.post(url, params=params)
    data = res.json()
    # 2. 응답 페이지에서 필요한 정보 추출
    if res.status_code == 200:
        return data
    else:
        raise Exception(f"요청 실패. 응답코드: {res.status_code}")      

data = get_question_list()
datata = data['data']['faqList']['items']
question_arr = []
for i in datata :
    question_arr.append([i['question'], i['answer']['html']])

import pandas as pd
faq = pd.DataFrame(question_arr, columns=["question" , "answer"])
faq.columns = ['Questions', 'Answer']

faq.to_csv('faq.csv',index=False)

In [9]:
#db 생성
import pandas as pd
! pip install pandas sqlalchemy pymysql
from sqlalchemy import create_engine

# CSV 파일 불러오기
csv_file_path = 'faq.csv'  # CSV 파일 경로
df = pd.read_csv(csv_file_path)

# MySQL 데이터베이스 연결 설정
username = 'scott'  # MySQL 사용자 이름
password = 'tiger'  # MySQL 비밀번호
host = 'localhost'  # MySQL 서버 주소 (보통 localhost)
port = '3306'  # MySQL 포트 (기본값은 3306)
database = 'faq'  # 데이터베이스 이름

# MySQL 데이터베이스 연결 문자열 생성
connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string)

# DataFrame을 MySQL 데이터베이스에 저장
df.to_sql('faq_table', engine, if_exists='replace', index=False)

print("CSV 파일이 MySQL 데이터베이스에 저장되었습니다.")

CSV 파일이 MySQL 데이터베이스에 저장되었습니다.


In [43]:
#JEJU
#https://jejuevservice.com/echarger

import requests
from bs4 import BeautifulSoup

url = "https://tago.kr/charge/faq.htm"

# questions
# a_selector = "div > div.contentList > div"
a_selector = "div.title > div > span"

# user-agent
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'

#subPage > div > div > div.contentList > div > div.faq_title
#subPage > div > div > div.contentList > div

def get_faq_list():
   
    # 1. 요청
    res = requests.get(url, headers={"user-agent":user_agent})
    # 2. 응답 페이지에서 필요한 정보 추출
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, "lxml")
        a_list = soup.select(a_selector)# [<a href="url">제목</a>, ...]
        result_list = []
        print(a_list)
        for a_tag in a_list:
            title = a_tag.get_text() 
            link = a_tag.get("href") 
            result_list.append([title.strip(), link])
        
        
        return result_list
    else:
        raise Exception(f"요청 실패. 응답코드: {res.status_code}")

result = get_faq_list()
print(result)

[<span class="table-cell">차량별 급속충전기 타입을 모르겠어요.</span>, <span class="table-cell">급속충전기 사용법이 궁금해요.</span>, <span class="table-cell">완속충전기 사용법이 궁금해요.</span>, <span class="table-cell">충전기 위치는 어떻게 확인하나요?</span>, <span class="table-cell">(SM3) 차량에서 커넥터가 안 빠져요.</span>, <span class="table-cell">충전기 커넥터 도어가 열리지 않아요.</span>, <span class="table-cell">충전기 화면에 비상정지라고 쓰여 있어요.</span>, <span class="table-cell">현재 충전하는 사람이 없는데 충전기화면에는 충전중이라고 나와요.</span>, <span class="table-cell">아이오닉인데 커넥터가 맞지 않아요</span>, <span class="table-cell">충전을 할 때마다 주행가능 거리가 다른데 차량에 문제가 있는건가요?</span>]
[['차량별 급속충전기 타입을 모르겠어요.', None], ['급속충전기 사용법이 궁금해요.', None], ['완속충전기 사용법이 궁금해요.', None], ['충전기 위치는 어떻게 확인하나요?', None], ['(SM3) 차량에서 커넥터가 안 빠져요.', None], ['충전기 커넥터 도어가 열리지 않아요.', None], ['충전기 화면에 비상정지라고 쓰여 있어요.', None], ['현재 충전하는 사람이 없는데 충전기화면에는 충전중이라고 나와요.', None], ['아이오닉인데 커넥터가 맞지 않아요', None], ['충전을 할 때마다 주행가능 거리가 다른데 차량에 문제가 있는건가요?', None]]


In [35]:
#타고 Question
#https://tago.kr/charge/faq.htm

import requests
from bs4 import BeautifulSoup


url = "https://tago.kr/charge/faq.htm"

# questions
# a_selector = "div > div.contentList > div"
a_selector = "#container > div > div.charge-info-floor1 > div > h4> span"

# user-agent
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'

#subPage > div > div > div.contentList > div > div.faq_title
#subPage > div > div > div.contentList > div

def get_faq_list():
   
    # 1. 요청
    res = requests.get(url, headers={"user-agent":user_agent})
    # 2. 응답 페이지에서 필요한 정보 추출
    if res.status_code == 200:
        soup = BeautifulSoup(res.content, "html.parser", from_encoding='utf-8')
        a_list = soup.select(a_selector)# [<a href="url">제목</a>, ...]
        result_list = []
        print(a_list)
        for a_tag in a_list:
            title = a_tag.get_text() 
            link = a_tag.get("href") 
            result_list.append([title.strip(), link])
        
        
        return result_list
    else:
        raise Exception(f"요청 실패. 응답코드: {res.status_code}")

result = get_faq_list()
print(result)


from pprint import pprint
result = get_faq_list()
pprint(result)

if __name__ == "__main__":
    result = get_faq_list()
    # from pprint import pprint
    # pprint(result)
    
    # DataFrame 
    result_df = pd.DataFrame(result, columns=["Question", ""])
    # print(result_df)
    result_df = result_df.drop("", axis = 1)
    
    # csv 파일 
    result_df.to_csv('tago_q.csv', index=False)

[<span>한 달에 주행거리가 1,000km라면 충전요금은 얼마나 나올까요?</span>, <span>아파트에 있는 충전기의 요금은 어떻게 되어 있나요?</span>, <span>긴급 충전을 위해 유류 비상 발전기를 싣고 다니면 안되나요?</span>, <span>완속충전이 좋나요, 급속충전이 좋나요?</span>, <span>과금형 콘센트와 태그형 충전기의 차이는 무엇인가요?</span>, <span>전기차 소모품은 내연차와 무엇이 다른가요?</span>, <span>히트펌프와 배터리 히팅 시스템은 무엇이고 옵션 추가해야 할까요?</span>, <span>고속도로 통행료 할인은 꼭 하이패스를 거쳐야만 하나요?</span>, <span>전기차 타기 위해 발급해야 하는 카드의 종류는 어떻게 되나요?</span>, <span>신용카드 중에 충전요금이 할인이 되는 카드가 있나요?</span>, <span>내연기관차는 고속주행 연비가 좋은데 왜 전기차는 시내주행 전비가 좋은가요?</span>]
[['한 달에 주행거리가 1,000km라면 충전요금은 얼마나 나올까요?', None], ['아파트에 있는 충전기의 요금은 어떻게 되어 있나요?', None], ['긴급 충전을 위해 유류 비상 발전기를 싣고 다니면 안되나요?', None], ['완속충전이 좋나요, 급속충전이 좋나요?', None], ['과금형 콘센트와 태그형 충전기의 차이는 무엇인가요?', None], ['전기차 소모품은 내연차와 무엇이 다른가요?', None], ['히트펌프와 배터리 히팅 시스템은 무엇이고 옵션 추가해야 할까요?', None], ['고속도로 통행료 할인은 꼭 하이패스를 거쳐야만 하나요?', None], ['전기차 타기 위해 발급해야 하는 카드의 종류는 어떻게 되나요?', None], ['신용카드 중에 충전요금이 할인이 되는 카드가 있나요?', None], ['내연기관차는 고속주행 연비가 좋은데 왜 전기차는 시내주행 전비가 좋은가요?', None]]
[<span>한 달에 주행거리가 1,000km

In [36]:
#타고 Answer
#https://jejuevservice.com/echarger

import requests
from bs4 import BeautifulSoup


url = "https://tago.kr/charge/faq.htm"

# questions
# a_selector = "div > div.contentList > div"
a_selector = "#container > div > div.charge-info-floor1 > div > div"

# user-agent
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'

#subPage > div > div > div.contentList > div > div.faq_title
#subPage > div > div > div.contentList > div

def get_faq_list():
   
    # 1. 요청
    res = requests.get(url, headers={"user-agent":user_agent})
    # 2. 응답 페이지에서 필요한 정보 추출
    if res.status_code == 200:
        soup = BeautifulSoup(res.content, "html.parser", from_encoding='utf-8')
        a_list = soup.select(a_selector)# [<a href="url">제목</a>, ...]
        result_list = []
        print(a_list)
        for a_tag in a_list:
            title = a_tag.get_text() 
            link = a_tag.get("href") 
            result_list.append([title.strip(), link])
        
        
        return result_list
    else:
        raise Exception(f"요청 실패. 응답코드: {res.status_code}")

result = get_faq_list()
print(result)


from pprint import pprint
result = get_faq_list()
pprint(result)

if __name__ == "__main__":
    result = get_faq_list()
    # from pprint import pprint
    # pprint(result)
    
    # DataFrame 
    result_df = pd.DataFrame(result, columns=["Answer", ""])
    # print(result_df)
    result_df = result_df.drop("", axis = 1)
    
    # csv 파일 
    result_df.to_csv('tago_a.csv', index=False)

[<div class="charge-box noml">
<div>
<div class="cgb-text">충전요금 = (주행거리 / 차량의 전비) * 1kWh당 충전비
							<br/>※ 환경부 급속공용, 완속 비공용기준
							<br/>
<br/>급속 = 292.9원/kWh
							<br/>완속 = 85원/kWh
							<br/>차량전비 = 평균 5.5km/kWh
							<br/>
<br/>급속 (1,000 / 5.5) * 292.9 = 53,250(원)
							<br/>완속 (1,000 / 5.5) * 85 = 15,450(원)
						</div>
</div>
</div>, <div class="charge-box noml">
<div>
<ul class="charge-box-list mt-style">
<li>아파트에 설치된 충전기는 설치 및 운영 주체에 따라 부과 방식과 요금체계가 다릅니다.</li>
<li>한전, 지엔텔, KT 등이 관리할 경우 회사 로고가 찍혀 있거나 해당 회사의 안내문이 부착되어 있습니다. 예를 들어 KEPCO가 찍혀 있으면 한전 APT 요금을 보시면 되고, KT, 기가차지(GigaCharge), 대영채비 등 충전기 회사가 써 있으면 해당 충전기 회사 홈페이지에 공지된 요금체계표를 따릅니다.</li>
<li>아파트 시공사 등이 자체 설치해서 관리사무실이 운영하는 충전기를 쓸 경우 아파트 입주자대표회의에서 정한 요금으로, 관리비에 별도 항목으로 부과됩니다. </li>
<li>아파트 주차장에 있는 일반 콘센트에 휴대용 충전기를 허락 없이 꽂으면 절도죄가 됩니다. "파워큐브", "이볼트" 등의 전기차 충전용 콘센트 운영 업체가 해당 콘센트를 전환 공사한 상태일 경우 해당 업체의 전용 충전기를 구입하여 사용할 수는 있습니다. 별도의 요금체계를 기반으로 해당 업체에서 요금을 청구합니다.</li>
</ul>
</div>
</div>, <div class="charge-box noml">
<div>
<div 

In [37]:
#Q&A merge

import pandas as pd

# CSV 파일
df1 = pd.read_csv('tago_a.csv')  
df2 = pd.read_csv('tago_q.csv')

# print(df1.index)
# print(df2.index)

merged_df = pd.concat([df2, df1], axis=1)

# 병합된 데이터프레임 저장
merged_df.to_csv('tago_faq.csv', index=False)

# 병합된 데이터 출력
# print(merged_df)

In [13]:
#GS
#https://www.gscev.com/kr/community/faq/3?searchWord=&searchCate=

! pip install selenium

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# 1. 웹 드라이버 (ChromeDrive)
driver = webdriver.Chrome()  # 자신의 chromedriver 경로 입력

# 2. 타겟 페이지
driver.get('https://www.gscev.com/kr/community/faq/3?searchWord=&searchCate=') 

# 3. 모든 질문 요소를 찾아서 클릭하여 답변을 가져오는 과정
# questions = driver.find_elements(By.CLASS_NAME, 'tit')
div = driver.find_elements(By.CLASS_NAME, '.faq-content.ui-accordion')
find_elements(By.CLASS_NAME, 'tit')

faq_data = []

for question in questions:
    question.click()  # 질문 클릭 (자바스크립트 동작)
    time.sleep(1)  # 잠시 대기 (동적 콘텐츠가 로드되기까지 기다림)

    # 답변 요소를 찾아서 텍스트 추출
    answer = driver.find_elements(By.CLASS_NAME, 'editor-reset')
    for i in answer:
        faq_data.append({
            'question': question.text,
            'answer': answer[0].text
        })


# 4. 웹 드라이버 종료
driver.quit()


# 5. 결과 출력 또는 저장
for item in faq_data:
    print(f"Question: {item['question']}")
    print(f"Answer: {item['answer']}")
    print()




# import requests
# from bs4 import BeautifulSoup

# url = "https://www.gscev.com/kr/community/faq/1?searchWord=&searchCate="

# # questions
# # a_selector = "div > div.contentList > div"
# # a_selector = div.content-info > div > div.faq-content.ui-accordion > div > div.accordion-item
# a_selector = "#faqForm > div.content-info > div > div.faq-content.ui-accordion > div > div > button > span > span"

# # user-agent
# user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'

# #subPage > div > div > div.contentList > div > div.faq_title
# #subPage > div > div > div.contentList > div

# def get_faq_list():
   
#     # 1. 요청
#     res = requests.get(url, headers={"user-agent":user_agent})
#     # 2. 응답 페이지에서 필요한 정보 추출
#     if res.status_code == 200:
#         soup = BeautifulSoup(res.text, "lxml")
#         a_list = soup.select(a_selector)# [<a href="url">제목</a>, ...]
#         result_list = []
#         print(a_list)
#         for a_tag in a_list:
#             title = a_tag.get_text() 
#             link = a_tag.get("href") 
#             result_list.append([title.strip(), link])
        
        
#         return result_list
#     else:
#         raise Exception(f"요청 실패. 응답코드: {res.status_code}")

# result = get_faq_list()
# print(result)

# #faqForm > div.content-info > div > div.faq-content.ui-accordion > div span

InvalidSelectorException: Message: invalid selector
from javascript error: {"status":32,"value":"An invalid or illegal selector was specified"}
  (Session info: chrome=129.0.6668.101); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalid-selector-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7758AB095+29557]
	(No symbol) [0x00007FF77581FA50]
	(No symbol) [0x00007FF7756DB56A]
	(No symbol) [0x00007FF7756E213E]
	(No symbol) [0x00007FF7756E44B7]
	(No symbol) [0x00007FF7756E4570]
	(No symbol) [0x00007FF77572F20F]
	(No symbol) [0x00007FF77572F8EC]
	(No symbol) [0x00007FF77577B7A7]
	(No symbol) [0x00007FF7757571CF]
	(No symbol) [0x00007FF77577851C]
	(No symbol) [0x00007FF775756F33]
	(No symbol) [0x00007FF77572116F]
	(No symbol) [0x00007FF7757222D1]
	GetHandleVerifier [0x00007FF775BDC96D+3378253]
	GetHandleVerifier [0x00007FF775C28497+3688311]
	GetHandleVerifier [0x00007FF775C1D1CB+3642539]
	GetHandleVerifier [0x000